In [1]:
# Interpolating WACCM data onto magnetic latitude 

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt 
from tqdm import tqdm

# WACCM Occ Freqs (Lat/Lon) - Data Loading

In [3]:

win_Monthfolderstr = 'Dec-Feb'
win_ds_months = ['12','01','02']

spr_Monthfolderstr = 'Mar-May'
spr_ds_months = ['03','04','05']

sum_Monthfolderstr = 'Jun-Aug'
sum_ds_months = ['06','07','08']

aut_Monthfolderstr = 'Sep-Nov'
aut_ds_months = ['09','10','11']


#run_name = 'Jianfei_run'
run_name = 'Wuhu_IonTr_run'
#run_name = 'Wuhu_IonTr_run_No_glbavg'

win_file = f'Nc_Files/SpE_Output/{run_name}_SpE_Output_{win_Monthfolderstr}.nc'
spr_file = f'Nc_Files/SpE_Output/{run_name}_SpE_Output_{spr_Monthfolderstr}.nc'
sum_file = f'Nc_Files/SpE_Output/{run_name}_SpE_Output_{sum_Monthfolderstr}.nc'
aut_file = f'Nc_Files/SpE_Output/{run_name}_SpE_Output_{aut_Monthfolderstr}.nc'

ds_months_ar = np.arange(0,3)   

ds_win = xr.open_dataset(win_file)
ds_spr = xr.open_dataset(spr_file)
ds_sum = xr.open_dataset(sum_file)
ds_aut = xr.open_dataset(aut_file)

ds_sum

<xarray.Dataset>
Dimensions:                    (lev: 126, lev_sl: 19, time: 336, lat: 96,
                                latsl: 36, lon: 144, LT: 48, LT_L: 144,
                                timesl: 2, mth: 3)
Coordinates:
  * lev                        (lev) float64 4.055e-10 5.207e-10 ... 970.6 992.6
  * lev_sl                     (lev_sl) float64 1.473e-05 1.891e-05 ... 0.001329
  * time                       (time) float64 0.0 1.0 2.0 ... 333.0 334.0 335.0
  * lat                        (lat) float64 -90.0 -88.11 -86.21 ... 88.11 90.0
  * latsl                      (latsl) float64 -87.5 -82.5 -77.5 ... 82.5 87.5
  * lon                        (lon) float64 0.0 2.5 5.0 ... 352.5 355.0 357.5
  * LT                         (LT) float64 0.0 0.5 1.0 1.5 ... 22.5 23.0 23.5
  * LT_L                       (LT_L) float64 0.0 0.1667 0.3333 ... 23.67 23.83
  * timesl                     (timesl) float64 0.0 1.0
  * mth                        (mth) float64 6.0 7.0 8.0
Data variables: (12/27)
    Zavg_sl                    (lev_sl) float64 ...
    altavg                     (lev) float64 ...
    altavg_sl                  (lev_sl) float64 ...
    times_str_min              (mth, timesl) object ...
    times_str_max              (mth, timesl) object ...
    SpEs_Occ_Fr_b_dsavglevLT   (lat) float64 ...
    ...                         ...
    Mptdensns                  (lev_sl, time, lat, lon, timesl, mth) float64 ...
    Mptdens_avv1_b_5d          (lev_sl, latsl, timesl, mth) float64 ...
    Mptdens_std                (lev_sl, lat, LT_L, timesl, mth) float64 ...
    Mptdens_avg                (lev_sl, lat, LT_L, timesl, mth) float64 ...
    SpEs_freq_time             (lev_sl, lat, LT_L, timesl, mth) float64 ...
    SpEsns_freq_time           (lev_sl, lat, lon, timesl, mth) float64 ...

# WACCM Raw Files (ALATM/ALONM)- Data Loading

In [4]:
file1name='Nc_Files/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h2.0001-03-*.nc'
ds = xr.open_mfdataset(file1name)
#ds

# WACCM - MagLat Interpolation

In [ ]:

ALATM = ds['ALATM']     #time,lat,lon
avALATM = np.mean(ALATM, axis=0)   # Average over time dimension -> [lat,lon]

lat_min = -75  # Replace with your minimum latitude
lat_max = 75   # Replace with your maximum latitude

latsl = ds_spr['lat'].sel(lat=slice(lat_min, lat_max))
nlat = len(latsl)
nlats = np.arange(0,nlat)

nlons = np.arange(0,144)
n = np.zeros((nlat, 144 ))

# Define seasons and corresponding datasets
seasons = ['spr', 'sum', 'aut', 'win']
datasets = [ds_spr, ds_sum, ds_aut, ds_win]

# Initialize arrays for each season
n_values = {season: np.zeros((nlat, nlons.size)) for season in seasons}
SpEsns_Occ_Fr_maglat = {season: np.zeros(nlat) for season in seasons}

# Loop through seasons
for season, dataset in zip(seasons, datasets):
    # Select the relevant dataset for the current season
    ds_season = dataset['SpEsns_Occ_Fr_dsavglev'].sel(lat=slice(lat_min, lat_max))
    
    # Loop through latitudes with a progress bar
    for ilat in tqdm(nlats, desc=f'Processing {season} data', unit='latitude'):
        for ilon in nlons:
            fp = ds_season[:, ilon].values    # Occurrence frequency values (lat,lon)
            xp = avALATM.sel(lat=slice(lat_min, lat_max))[:, ilon].values     #mag lat values corresponding to each occ freq above (not on an evenly spaced grid)
            x = latsl[ilat]
            n_values[season][ilat, ilon] = np.interp(x, xp, fp)

    # Calculate mean along the longitude axis
    SpEsns_Occ_Fr_maglat[season] = np.mean(n_values[season], axis=1)


In [ ]:
########### Save to nc file ##########

dataset.set_index

# Create an empty xarray Dataset to store variables
result_dataset = xr.Dataset()

# Loop through seasons
for season, values in SpEsns_Occ_Fr_maglat.items():
    
    result_dataarray_occ_freq = xr.DataArray(SpEsns_Occ_Fr_maglat[season], coords={'mlat': latsl.values}, 
                                             dims=['mlat'], 
                                             name=f'{season}_Occ_Fr_maglat', 
                                             attrs={'Data': 'Occ Freq interp onto mag lat, avgd over lon dim (~avgd at each constant mag lat)', 'Dims': 'mlat'})
    
    # Add the DataArray to the Dataset
    result_dataset[f'{season}_Occ_Fr_maglat'] = result_dataarray_occ_freq

    # Access the corresponding n_values for the current season
    result_dataarray_n_values = xr.DataArray(n_values[season], coords={'mlat': latsl.values, 'lon': ds['lon'].values}, 
                                             dims=['mlat', 'lon'], 
                                             name=f'{season}_Occ_Fr_maglatlon', 
                                             attrs={'Data': 'Occ Freq interp onto mag lat', 'Dims': 'mlat & lon'})
    
    # Add the DataArray to the Dataset
    result_dataset[f'{season}_Occ_Fr_maglatlon'] = result_dataarray_n_values
    

output_directory = "./Nc_Files/SpE_Output/Mag_Coords/"
output_file = "Occ_Fr_maglat_lon.nc"
result_dataset.to_netcdf(f'{output_directory}{output_file}')

# OLD CODE - IGNORE ME

In [ ]:
#      Load WACCM Data #
#===============================

ds_Occ_Fr_maglat_maglatlon = xr.open_dataset('./Nc_Files/SpE_Output/Mag_Coords/Occ_Fr_maglat_lon.nc')
mlat_w = ds_Occ_Fr_maglat_maglatlon['mlat']

spr_Occ_Fr_maglat_w = ds_Occ_Fr_maglat_maglatlon['spr_Occ_Fr_maglat']
sum_Occ_Fr_maglat_w = ds_Occ_Fr_maglat_maglatlon['sum_Occ_Fr_maglat']
aut_Occ_Fr_maglat_w = ds_Occ_Fr_maglat_maglatlon['aut_Occ_Fr_maglat']
win_Occ_Fr_maglat_w = ds_Occ_Fr_maglat_maglatlon['win_Occ_Fr_maglat']

In [ ]:
#========================================
# WACCM - MagLon Interpolation (not working!)
#========================================

In [ ]:
########### L O N interpolation ##########

ALONM = ds['ALONM']     #time,lat,lon
avALONM = np.mean(ALONM, axis=0)
avALONMsel = avALONM.sel(lat=slice(lat_min, lat_max))
avALONMsh = avALONMsel.roll(lon=144// 2, roll_coords=True)

lonsh = (lon + 180) % 360 - 180    #0-360 to -180 - 180
lonshh = lonsh.roll(lon=144// 2)



lat_min = -75  # Replace with your minimum latitude
lat_max = 75   # Replace with your maximum latitude


mlatsl = ds_Occ_Fr_maglat_maglatlon['mlat']
nmlat = len(mlatsl)
nmlats = np.arange(0,nlat)

nlons = np.arange(0,144)
n = np.zeros((nmlat, 144 ))



#Define seasons and corresponding datasets
seasons = ['spr_Occ_Fr_maglatlon', 'sum_Occ_Fr_maglatlon', 'aut_Occ_Fr_maglatlon', 'win_Occ_Fr_maglatlon']


# Initialize arrays for each season
m_values = {season: np.zeros((nlat, nlons.size)) for season in seasons}


# Loop through seasons
for season in seasons:
    # Select the relevant dataset for the current season
    ds_season = ds_Occ_Fr_maglat_maglatlon[season]
    ds_seasonsh = ds_season.roll(lon=144// 2, roll_coords=True)
    
 
    for ilon in tqdm(nlons, desc=f'Processing {season} data'):
        for ilat in nmlats:
            fp = ds_season[ilat, :].values
            xp = avALONMsel[ilat,:].values
            x = lonsh[ilon]
            m_values[season][ilat, ilon] = np.interp(x, xp, fp)


In [ ]:
########### Save to nc file ##########

seasons = ['spr_Occ_Fr_maglatlon', 'sum_Occ_Fr_maglatlon', 'aut_Occ_Fr_maglatlon', 'win_Occ_Fr_maglatlon']

dataset.set_index

# Create an empty xarray Dataset to store variables
result_dataset = xr.Dataset()

# Loop through seasons
for season, values in m_values.items():
    # Extract the prefix (e.g., 'spr', 'sum', 'aut', 'win') from the full season name
    season_prefix = season.split('_')[0]
    
    # Access the corresponding n_values for the current season
    result_dataarray_m_values = xr.DataArray(m_values[season], coords={'mlat': latsl.values, 'mlon': lonsh.values}, 
                                             dims=['mlat', 'mlon'], 
                                             name=f'{season_prefix}_Occ_Fr_maglatmaglon', 
                                             attrs={'Data': 'Occ Freq interp onto mag lat', 'Dims': 'mlat & lon'})
    
    # Add the DataArray to the Dataset
    result_dataset[f'{season_prefix}_Occ_Fr_maglatamglon'] = result_dataarray_m_values
    
output_directory = "./Nc_Files/SpE_Output/Mag_Coords/"
output_file = "Occ_Fr_maglatmaglon.nc"
result_dataset.to_netcdf(f'{output_directory}{output_file}')

In [ ]:
#========================================
# MagLat-MagLon Figure Plotting
#========================================

In [ ]:
ds_Occ_Fr_maglatmaglon = xr.open_dataset('./Nc_Files/SpE_Output/Mag_Coords/Occ_Fr_maglatmaglon.nc')
ds_Occ_Fr_maglatmaglon

In [ ]:
spr_Occ_Fr_maglatmaglon = ds_Occ_Fr_maglatmaglon['spr_Occ_Fr_maglatamglon']
sum_Occ_Fr_maglatmaglon = ds_Occ_Fr_maglatmaglon['sum_Occ_Fr_maglatamglon']
aut_Occ_Fr_maglatmaglon = ds_Occ_Fr_maglatmaglon['aut_Occ_Fr_maglatamglon']
win_Occ_Fr_maglatmaglon = ds_Occ_Fr_maglatmaglon['win_Occ_Fr_maglatamglon']

mlat = ds_Occ_Fr_maglatmaglon['mlat']
mlon = ds_Occ_Fr_maglatmaglon['mlon']

In [ ]:
####### Plot Occ_Fr_maglatmaglon ########


# # Create a 2D grid using meshgrid
# lon_2d, lat_2d = np.meshgrid(lon, lat)

fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
contour = ax.contourf(mlon, mlat, spr_Occ_Fr_maglatmaglon, cmap='jet'#, levels=np.linspace(0, 0.5, 11)
                      , transform=ccrs.PlateCarree())
cbar = plt.colorbar(contour
                    #, label=''
                    , ax=ax, orientation='vertical', pad=0.05, aspect=30, shrink=0.5)
ax.coastlines()

ax.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax.set_xlabel('Longitude')
ax.set_yticks(np.linspace(-90, 90, 7))
ax.set_ylabel('Latitude')
#ax.set_title(f'')

#gl = ax.gridlines(draw_labels=True, linestyle='--')

figname = f'.png'
#plt.savefig('./Figures/Factor/' + str(figname), dpi=400, bbox_inches='tight')

plt.show()